In [15]:
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain.llms.base import LLM
from langchain.chains import ConversationChain
from typing import Optional, List, Any
import faiss
import json
import numpy as np
from sentence_transformers import SentenceTransformer
import requests


In [16]:
embedder = SentenceTransformer("all-MiniLM-L6-v2")

In [17]:
def load_index_and_meta(index_path, meta_path):
    index = faiss.read_index(index_path)
    with open(meta_path, "r", encoding="utf-8") as f:
        meta = json.load(f)
    return index, meta

def search_faiss(query, index, meta, top_k=2):
    query_vec = embedder.encode([query])
    query_vec = np.array(query_vec).astype("float32")
    D, I = index.search(query_vec, top_k)
    results = []
    for idx in I[0]:
        if idx < len(meta):
            results.append(meta[idx])
    return results
class ollama(LLM):
    model_name: str = "mistral"

    @property
    def _llm_type(self) -> str:
        return "ollama"
    
    def _call(self,prompt: str, stop: Optional[List[str]] = None, **kwargs: Any):
        response = requests.post(
        "http://127.0.0.1:11434/api/generate",
        json={"model": self.model_name, "prompt": prompt, "stream": False}
    )
        return response.json()["response"]

In [18]:
ipc_index, ipc_meta = load_index_and_meta("ipc.index", "ipc_meta.json")
judg_index, judg_meta = load_index_and_meta("judgements.index", "judgements_meta.json")

In [ ]:
custom=PromptTemplate(input_variables=["history","input"],template=""" You are an legal Chatbot assistant specializing in India Penal Code.
                      
                      Previous conversations:{history}
                      
                      Current question and also judgements :{input}
                      
                    provide an suitable reply for the current question by utilizing previous conversations, keep it around 20 to 30 words .
                      Utilize similar case judgement and compare it with the user case summary and reply but do not repeat about judgement in every replies.
                      Only use these given text as knowledge, do not add your own knowledge.
                     Only talk about legal replies DO NOT REPLY ANY QUESTIONS WHICH ARE OUT OF INDIAN PENAL CODE .""")

In [ ]:
llm=ollama()
memory=ConversationBufferMemory()
conversion = ConversationChain(llm=llm,memory=memory,prompt=custom,verbose=False)

In [21]:
print("⚖️ IPC + Judgement Chatbot (type 'exit' to quit)\n")

user_input = input("You: ").strip()


ipc_results = search_faiss(user_input, ipc_index, ipc_meta)
judg_results = search_faiss(user_input, judg_index, judg_meta)

        # Step 2: Build context
context = "Relevant IPC Sections:\n"
for sec in ipc_results:
    context += f"- Section {sec['section_number']}: {sec['section_text']} (Punishment: {sec['punishment']})\n"

prompt = f"User Query: {user_input}\n\n{context}\n\nBased on the above IPC sections, Name the sections relating to the user query and also list possible punishments."
Reply_1=llm._call(prompt)

context += "\nRelevant Judgements:\n"
for case in judg_results:
    context += f"- {case['section']}: {case['text']}\n"



print(f"\nBot: {Reply_1}\n")

    


⚖️ IPC + Judgement Chatbot (type 'exit' to quit)


Bot:  Based on the user query, the following Indian Penal Code (IPC) sections could potentially apply:

1. Section 376 - Rape: This section criminalizes all forms of sexual intercourse without consent, including when the victim is under 18 years old or incapable of giving consent due to intoxication or unsoundness of mind. The penalty for rape includes imprisonment ranging from 7 years up to life and may also include a fine.

2. Section 342 - Wrongful confinement: This section penalizes the act of unlawfully confining a person against their will, which could potentially apply if the girl was forced or tricked into a situation where she became pregnant. The penalty for wrongful confinement is imprisonment up to two years, or fine, or both.

3. Section 317 - Abetment of an offence punishable with death or imprisonment for life or imprisonment: This section applies when a person abets or procures another person to commit any offense that 

In [1]:
while True:
    user_input = input("You: ").strip()
    if user_input.lower() in ["exit", "quit"]:
        print("Bot: Goodbye!")
        break

    user_input+=f"\n\n similar case judgements : {context}"

    answer = conversion.predict(input=user_input)

    print(f"\nBot: {answer}\n")

NameError: name 'context' is not defined